In [74]:
import projections
from importlib import reload
reload(projections)
projections.get_nf_d_projections()

{'Los Angeles Rams': 9.75,
 'Cincinnati': 9.16,
 'San Francisco': 8.84,
 'Buffalo': 8.06,
 'Cleveland': 7.61,
 'Kansas City': 7.45,
 'Tampa Bay': 7.35,
 'Denver': 7.34,
 'Atlanta': 7.22,
 'Seattle': 6.93,
 'Philadelphia': 6.84,
 'Chicago': 6.66,
 'Pittsburgh': 6.58,
 'Indianapolis': 6.49,
 'Minnesota': 6.41,
 'Jacksonville': 6.29,
 'Los Angeles Chargers': 6.16,
 'Washington': 6.03,
 'Detroit': 5.87,
 'Arizona': 5.53,
 'Tennessee': 5.44,
 'Carolina': 5.31,
 'New England': 5.17,
 'Green Bay': 5.05,
 'Dallas': 4.86,
 'New Orleans': 4.63,
 'New York Jets': 4.22,
 'Miami': 4.02,
 'New York Giants': 3.75,
 'Houston': 3.03}

In [75]:
#!usr/bin/python3

import sys
import projections
import pandas as pd
from itertools import combinations
import numpy as np
from joblib import Parallel, delayed
import datetime
from collections import defaultdict

def remove_suffix(p, fanduel=False):

    if fanduel:
        if p != 'Michael Pittman Jr.':
            p = p.replace(' Jr.', '')
        if fanduel_d.get(p.strip()):
            p = fanduel_d.get(p.strip())
    if (not fanduel) and defense.get(p.strip()):
        p = defense.get(p.strip())
    p = p.replace(' III', '')
    p = p.replace(' II', '')
    p = p.replace(' IV', '')
    if p != 'Las Vegas':
        p = p.replace(' V', '')
    p = p.replace('AJ ', 'A.J. ')
    p = p.replace('DJ Chark Jr.', 'D.J. Chark')
    p = p.replace('Marvin Jones Jr.', 'Marvin Jones')
    p = p.replace('Robert Griffin', 'Robert Griffin III')
    p = p.replace('Darrell Henderson Jr.', 'Darrell Henderson')
    p = p.replace('Odell Beckham Jr.', 'Odell Beckham')
    p = p.replace('DJ ', 'D.J. ')
    p = p.replace('DK ', 'D.K. ')
    p = p.replace('Eli Mitchell', 'Elijah Mitchell')

    return p

defense = {
     'Steelers': 'Pittsburgh',
     'Chargers': 'Los Angeles Chargers',
     'Browns': 'Cleveland',
     'Vikings': 'Minnesota',
     'WAS Football Team': 'Washington',
     'Dolphins': 'Miami',
     'Lions': 'Detroit',
     'Patriots': 'New England',
     'Ravens': 'Baltimore',
     'Buccaneers': 'Tampa Bay',
     'Saints': 'New Orleans',
     'Chiefs': 'Kansas City',
     'Bengals': 'Cincinnati',
     'Texans': 'Houston',
     'Broncos': 'Denver',
     'Eagles': 'Philadelphia',
     'Panthers': 'Carolina',
     'Cardinals': 'Arizona',
     'Packers': 'Green Bay',
     'Titans': 'Tennessee',
     'Cowboys': 'Dallas',
     'Rams': 'Los Angeles Rams',
     'Seahawks': 'Seattle',
     'Jaguars': 'Jacksonville',
     'Falcons': 'Atlanta',
     'Jets': 'New York Jets',
     'Colts': 'Indianapolis',
     'Raiders': 'Las Vegas',
     'Bills': 'Buffalo',
    'Bears': 'Chicago',
    '49ers': 'San Francisco',
    'Giants': 'New York Giants',
    
 }

fanduel_d = {
    'Pittsburgh Steelers': 'Pittsburgh',
    'Miami Dolphins': 'Miami',
    'Baltimore Ravens': 'Baltimore',
    'Cleveland Browns': 'Cleveland',
    'Los Angeles Chargers': 'Los Angeles Chargers',
    'Minnesota Vikings': 'Minnesota',
    'New England Patriots': 'New England',
    'Green Bay Packers': 'Green Bay',
    'New Orleans Saints': 'New Orleans',
    'Washington Football Team': 'Washington',
    'Indianapolis Colts': 'Indianapolis',
    'Philadelphia Eagles': 'Philadelphia',
    'Denver Broncos': 'Denver',
    'Tennessee Titans': 'Tennessee',
    'Cincinnati Bengals': 'Cincinnati',
    'Houston Texans': 'Houston',
    'Detroit Lions': 'Detroit',
    'Carolina Panthers': 'Carolina',
    'Jacksonville Jaguars': 'Jacksonville',
    'Atlanta Falcons': 'Atlanta',
    'New York Jets': 'New York Jets',
    'Dallas Cowboys': 'Dallas',
    'Los Angeles Rams': 'Los Angeles Rams',
    'San Francisco 49ers': 'San Francisco',
    'Seattle Seahawks': 'Seattle',
    'Chicago Bears': 'Chicago',
    'Tampa Bay Buccaneers': 'Tampa Bay',
    'Buffalo Bills': 'Buffalo',
    'Kansas City Chiefs': 'Kansas City',
    'Arizona Cardinals': 'Arizona',
}

plyr_dict = projections.get_nf_projections()
plyr_dict.update(projections.get_nf_d_projections())

In [76]:
# f = 'fanduel_thurs.csv'
f = 'fanduel.csv'
df = pd.read_csv('inputs/' + f)
df = df.drop(df.loc[(df['Nickname'] == 'Ryan Griffin') & (df.Team == 'TB')].index)
assert len(df) == len(df['Nickname'].unique())
players = df['Nickname'].to_list()
players = {p: remove_suffix(p, True) for p in players}
df['Nickname'] = df['Nickname'].replace(players)
df = df.set_index('Nickname')
df['Projection'] = pd.DataFrame.from_dict(plyr_dict, orient='index')

In [77]:
# df.groupby('Nickname').count().sort_values(by=['Position'], ascending=False)

In [78]:
# df.loc[df['Position']=='D']

In [79]:
# f = 'draftkings_thurs.csv'
f = 'draftkings.csv'
df = pd.read_csv('inputs/' + f)
players = df['Name'].to_list()
players = {p: remove_suffix(p) for p in players}
df['Name'] = df['Name'].replace(players)
df = df.drop(df.loc[(df['Name'] == 'Josh Johnson') & (df.TeamAbbrev == 'NYJ')].index)
df = df.drop(df.loc[(df['Name'] == 'Ryan Griffin') & (df.TeamAbbrev == 'TB')].index)
df = df.set_index('Name')
df['Projection'] = pd.DataFrame.from_dict(plyr_dict, orient='index')
df = df.reset_index()
# df.groupby('Name').count().sort_values(by=['Position'])
df['Name'] = df['Name'].apply(lambda x: x.strip())
assert len(df['Name']) == len(df['Name'].unique())
df.set_index(['Name'], inplace=True)

In [80]:
# df.loc[(df.Projection.isna()) & (df.FPPG > 0)].sort_values('Salary', ascending=False).head(20)
# df.loc[(df.Projection.isna()) & (df.AvgPointsPerGame > 0)].sort_values('AvgPointsPerGame', ascending=False).head(20)

In [81]:
# df.groupby('Name').count().sort_values(by=['Position'], ascending=False)

In [82]:
df = df[df['Projection'] > 1]

defense = {'d': 'DST', 'f': 'D'}
plyr_names = {'d': 'Name', 'f': 'Nickname'}
salary = {'d': 50000, 'f': 60000}

df = df.reset_index()
df_pos_salary = df.groupby(['Position', 'Salary'])['Projection'].agg([np.mean, np.max])
df_pos_salary = df_pos_salary.reset_index()
df = df.merge(df_pos_salary, on=['Position', 'Salary'])
df = df[((df['Position'].isin(['QB', defense[f[0]]])) & (df['Projection'] == df['amax'])) | ((df['Position'].isin(['WR', 'TE', 'RB'])) & (df['Projection'] == df['mean']))]
df.set_index(plyr_names[f[0]], inplace=True)
df = df.reset_index()
df_pos = df.groupby(['Position', 'Projection'])['Salary'].agg([np.min])['amin']
df_pos = df_pos.reset_index()
df = df.merge(df_pos, on=['Position', 'Projection'])
df = df[((df['Position'].isin(['QB', defense[f[0]]])) & (df['Salary'] <= df['amin'])) | (df['Position'].isin(['WR', 'TE', 'RB']))]
df.set_index(plyr_names[f[0]], inplace=True)

In [83]:
grouped = df.groupby(['Position'])
position_dict = defaultdict()
for pos, frame in grouped:
    position_dict[pos] = frame.to_dict(orient='index')

player_dict = defaultdict()
for item in position_dict.items():
    for plyr_name in item[1].keys():
        player_dict[plyr_name] = item[1][plyr_name]

te_plyr_list = [k for k,v in player_dict.items() if v['Position'] == 'TE']
rb_plyr_list = [k for k,v in player_dict.items() if v['Position'] == 'RB']
wr_plyr_list = [k for k,v in player_dict.items() if v['Position'] == 'WR']
singles_list = [[qb, d] for qb in position_dict['QB'].keys() for d in position_dict[defense[f[0]]].keys()]

def return_combos(plyr_list, count):
    return list(combinations(plyr_list, count))

te = return_combos(te_plyr_list, 2)
wr_3 = return_combos(wr_plyr_list, 3)
wr_4 = return_combos(wr_plyr_list, 4)
rb_2 = return_combos(rb_plyr_list, 2)
rb_3 = return_combos(rb_plyr_list, 3)

flex_combos = {
    1: {'TE': te_plyr_list, 'RB': rb_3, 'WR': wr_3},
    2: {'TE': te_plyr_list, 'RB': rb_2,  'WR': wr_4},
    3: {'TE': te, 'RB': rb_2, 'WR': wr_3},
    }

def df_value(row, k, n):
    if n == 1:
        return player_dict[row[0]][k]
    if n == 2:
        return player_dict[row[0]][k]+ player_dict[row[1]][k]
    if n == 3:
        return player_dict[row[0]][k] + player_dict[row[1]][k] + player_dict[row[2]][k]
    if n == 4:
        return player_dict[row[0]][k] + player_dict[row[1]][k] + player_dict[row[2]][k] + player_dict[row[3]][k]

def clean_frame(f):
    df_salary = f.groupby(['Salary'])['Projection'].agg([np.max])['amax']
    f = f.merge(df_salary.to_frame(), on=['Salary'])
    f = f[f['Projection'] >= f['amax']]
    del f['amax']

    df_pos = f.groupby(['Projection'])['Salary'].agg([np.min])['amin']
    f = f.merge(df_pos.to_frame(), on=['Projection'])
    f = f[f['Salary'] <= f['amin']]
    del f['amin']
    return f

def get_projections(ck):

    df_qb = pd.DataFrame(singles_list)
    col_n = len(df_qb.columns)
    df_qb['Salary'] = df_qb.apply(df_value, args=(['Salary', col_n]), axis=1)
    df_qb['Projection'] = df_qb.apply(df_value, args=(['Projection', col_n]), axis=1)
    df_qb = clean_frame(df_qb)

    df_te = pd.DataFrame(flex_combos[ck]['TE'])
    col_n = len(df_te.columns)
    df_te['Salary'] = df_te.apply(df_value, args=(['Salary', col_n]), axis=1)
    df_te['Projection'] = df_te.apply(df_value, args=(['Projection', col_n]), axis=1)
    df_te = clean_frame(df_te)

    df_rb = pd.DataFrame(flex_combos[ck]['RB'])
    col_n = len(df_rb.columns)
    df_rb['Salary'] = df_rb.apply(df_value, args=(['Salary', col_n]), axis=1)
    df_rb['Projection'] = df_rb.apply(df_value, args=(['Projection', col_n]), axis=1)
    df_rb = clean_frame(df_rb)

    df_wr = pd.DataFrame(flex_combos[ck]['WR'])
    col_n = len(df_wr.columns)
    df_wr['Salary'] = df_wr.apply(df_value, args=(['Salary', col_n]), axis=1)
    df_wr['Projection'] = df_wr.apply(df_value, args=(['Projection', col_n]), axis=1)
    df_wr = clean_frame(df_wr)

    df_qb['key'] = 1
    df_te['key'] = 1
    df_rb['key'] = 1
    df_wr['key'] = 1

    df = df_te.merge(df_rb, on=['key'], suffixes=('_te', '_rb'))
    df['Salary'] = df['Salary_te'] + df['Salary_rb']
    df['Projection'] = df['Projection_te'] + df['Projection_rb']
    df = df[df['Salary'] <= salary[f[0]] - (min(df_wr['Salary']) + min(df_qb['Salary']))]
    df = clean_frame(df)

    df = df.merge(df_wr, on=['key'], suffixes=('_2', '_wr'))
    df['Salary'] = df['Salary_2'] + df['Salary_wr']
    df['Projection'] = df['Projection_2'] + df['Projection_wr']
    df = df[df['Salary'] <= salary[f[0]] - min(df_qb['Salary'])]
    df = clean_frame(df)

    df = df.merge(df_qb, on=['key'], suffixes=('_3', '_qb'))
    df['Salary'] = df['Salary_3'] + df['Salary_qb']
    df['Projection'] = df['Projection_3'] + df['Projection_qb']
    df = df[df['Salary'] <= salary[f[0]]]
    df = clean_frame(df)

    df = df.sort_values('Projection', ascending=False)
    df.columns = [str(x) for x in df.columns]
    df = df[df.columns[~df.columns.str.contains('Projection_|Salary_|key')]].dropna(axis=1)
    df.columns = list(range(1,10)) + ['Salary', 'Projection']
    return df

In [84]:
# results = Parallel(n_jobs=-1)(delayed(get_projections)(k) for k in flex_combos.keys())
# df[list(range(1,10)) + ['Salary', 'Projection']].sort_values('Projection', ascending=False)

In [85]:
# if __name__=="__main__":
start_time = datetime.datetime.now()
results = Parallel(n_jobs=-1)(delayed(get_projections)(k) for k in flex_combos.keys())
df = pd.concat(results, sort=True)
df.reset_index(inplace=True)
df = df[list(range(1,10)) + ['Salary', 'Projection']].sort_values('Projection', ascending=False)
# df.to_csv('results/output_' + sys.argv[1])
df.to_csv('results/output_' + f)

print (datetime.datetime.now() - start_time)
print (df.iloc[0:10])

0:00:01.511772
                  1              2                  3                  4  \
0    Dallas Goedert     Nick Chubb  Darrell Henderson      Chuba Hubbard   
1        Jared Cook  D'Andre Swift         Nick Chubb  Darrell Henderson   
139      Jared Cook  D'Andre Swift  Darrell Henderson         A.J. Brown   
2        Jared Cook  D'Andre Swift  Darrell Henderson      Chuba Hubbard   
140  Dallas Goedert  D'Andre Swift  Darrell Henderson      Brandin Cooks   
3        Jared Cook  D'Andre Swift  Darrell Henderson      Chuba Hubbard   
4        Jared Cook  D'Andre Swift  Darrell Henderson      Chuba Hubbard   
5        Jared Cook  D'Andre Swift  Darrell Henderson      Chuba Hubbard   
141      Jared Cook     Nick Chubb  Darrell Henderson         A.J. Brown   
295  Dallas Goedert     Jared Cook      D'Andre Swift  Darrell Henderson   

                 5              6              7            8             9  \
0    DeVonta Smith    Tee Higgins  Jakobi Meyers   Josh Allen    Wash

In [86]:
# df.loc[df[8]=='Tua Tagovailoa']
df.iloc[0]

1                Dallas Goedert
2                    Nick Chubb
3             Darrell Henderson
4                 Chuba Hubbard
5                 DeVonta Smith
6                   Tee Higgins
7                 Jakobi Meyers
8                    Josh Allen
9                    Washington
Salary                    50000
Projection               138.74
Name: 0, dtype: object

In [48]:
plyr_dict['Chicago']

6.66

In [49]:
plyr_dict['Detroit']

5.87